1、准备数据，构建模型

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader=DataLoader(training_data, batch_size=64)
test_dataloader=DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self, x):
            x=self.flatten(x)
            logits=self.linear_relu_stack(x)
            return logits
model = NeuralNetwork()

2、设置超参数的值

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

3、设置目标函数

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

4、构建优化器，对参数进行更新

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

(1)、每一步训练之前都要对优化器进行置零操作，optimizer.zero_grad();
(2)、计算导数，loss.backward();
(3)、根据优化器对每个参数进行更新，optimizer.step()

5、完整实现

In [12]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        # Compute prediction and loss
        pred=model(X)
        loss=loss_fn(pred,y)
        
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss,current=loss.item(),batch*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    test_loss, correct=0,0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred=model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

epochs=10
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------
loss:2.154055 [    0/60000]
loss:2.143764 [ 6400/60000]
loss:2.088007 [12800/60000]
loss:2.109848 [19200/60000]
loss:2.044537 [25600/60000]
loss:1.986367 [32000/60000]
loss:2.027668 [38400/60000]
loss:1.939307 [44800/60000]
loss:1.947193 [51200/60000]
loss:1.875023 [57600/60000]
Test Error: 
 Accuracy: 50.6%, Avg loss: 1.871184 

Epoch 2
------------------------------
loss:1.901496 [    0/60000]
loss:1.869886 [ 6400/60000]
loss:1.754446 [12800/60000]
loss:1.803187 [19200/60000]
loss:1.674495 [25600/60000]
loss:1.635358 [32000/60000]
loss:1.669759 [38400/60000]
loss:1.565897 [44800/60000]
loss:1.600654 [51200/60000]
loss:1.494141 [57600/60000]
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.508971 

Epoch 3
------------------------------
loss:1.574825 [    0/60000]
loss:1.538503 [ 6400/60000]
loss:1.392948 [12800/60000]
loss:1.467815 [19200/60000]
loss:1.342461 [25600/60000]
loss:1.345086 [32000/60000]
loss:1.365377 [38400/60000]
loss:1.286568 [44800/60

*数据集、模型、训练*